In [5]:
import pandas as pd
import unicodedata
import numpy as np

# 1. Definir la función de limpieza (¡Necesaria para la columna 'barrio_limpio'!)
def limpiar_texto(s):
    """Estandariza el texto: mayúsculas, sin tildes, sin espacios iniciales/finales."""
    if isinstance(s, str):
        s = s.strip().upper()
        # Normaliza y elimina tildes/caracteres diacríticos
        s = ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')
        return s
    return s

# 2. CARGAR LOS DATAFRAMES (DEFINICIÓN DE LAS VARIABLES)
# Si no ejecutas estas dos líneas, Python no sabrá qué son df_properties y df_barrios_ref.
df_properties = pd.read_csv('properties.csv')
df_barrios_ref = pd.read_csv('barrios_bogota_localidades_limpio2.csv')

# 3. PREPARACIÓN DE LAS LLAVES
# Crear la columna de unión limpia en ambas bases
df_properties['barrio_limpio'] = df_properties['barrio'].apply(limpiar_texto)
df_barrios_ref['barrio_limpio'] = df_barrios_ref['Barrio'].apply(limpiar_texto)

# Eliminar duplicados en la base de referencia para el merge
df_barrios_ref_unique = df_barrios_ref[['Localidad', 'UPZ', 'barrio_limpio']].drop_duplicates(subset=['barrio_limpio'])


# 4. EJECUTAR EL MERGE (UNION)
df_merged = df_properties.merge(
    df_barrios_ref_unique,
    left_on='barrio_limpio',
    right_on='barrio_limpio',
    how='left'
)

print("¡Merge completado con éxito!")
print(f"Propiedades después del merge: {len(df_merged)}")
print(df_merged[['barrio', 'barrio_limpio', 'Localidad', 'UPZ']].head())

¡Merge completado con éxito!
Propiedades después del merge: 585
        barrio barrio_limpio  Localidad          UPZ
0  PARDO RUBIO   PARDO RUBIO  Chapinero  Pardo Rubio
1      USAQUEN       USAQUEN        NaN          NaN
2   LOS CEDROS    LOS CEDROS    Usaquén   Los Cedros
3     EL PRADO      EL PRADO        NaN          NaN
4      TOBERIN       TOBERIN        NaN          NaN


In [6]:
# A. Limpieza en la base de Propiedades
df_properties['barrio_limpio'] = df_properties['barrio'].apply(limpiar_texto)

# B. Limpieza en la base de Referencia
df_barrios_ref['barrio_limpio'] = df_barrios_ref['Barrio'].apply(limpiar_texto)

# C. Seleccionar y limpiar la tabla de referencia
# Eliminar duplicados en la llave 'barrio_limpio', manteniendo solo la información necesaria.
df_barrios_ref_unique = df_barrios_ref[['Localidad', 'UPZ', 'barrio_limpio']].drop_duplicates(subset=['barrio_limpio'])

In [7]:
# Realizar el Left Merge usando el campo limpio como llave
df_merged = df_properties.merge(
    df_barrios_ref_unique,
    left_on='barrio_limpio',      # Llave de df_properties
    right_on='barrio_limpio',     # Llave de df_barrios_ref
    how='left'                    # Mantiene todas las filas de df_properties
)

print("¡Merge completado!")
print(f"Número total de filas en el DataFrame final: {len(df_merged)}")

# Mostrar las nuevas columnas para verificación
print("\nVerificación de las nuevas columnas:")
print(df_merged[['barrio', 'barrio_limpio', 'Localidad', 'UPZ']].head())

¡Merge completado!
Número total de filas en el DataFrame final: 585

Verificación de las nuevas columnas:
        barrio barrio_limpio  Localidad          UPZ
0  PARDO RUBIO   PARDO RUBIO  Chapinero  Pardo Rubio
1      USAQUEN       USAQUEN        NaN          NaN
2   LOS CEDROS    LOS CEDROS    Usaquén   Los Cedros
3     EL PRADO      EL PRADO        NaN          NaN
4      TOBERIN       TOBERIN        NaN          NaN


In [8]:
# Contar el número de propiedades sin localidad asignada
propiedades_sin_localidad = df_merged['Localidad'].isna().sum()

print(f"\nNúmero de propiedades sin Localidad asignada (NaN): {propiedades_sin_localidad}")
print(f"El porcentaje de datos sin unión es: {(propiedades_sin_localidad / len(df_merged) * 100):.2f}%")

# Opcional: ver los nombres de los barrios originales que fallaron
barrios_faltantes_frecuentes = df_merged[df_merged['Localidad'].isna()]['barrio'].value_counts().head(5)
print("\nTop 5 barrios originales que NO coincidieron:")
print(barrios_faltantes_frecuentes)


Número de propiedades sin Localidad asignada (NaN): 304
El porcentaje de datos sin unión es: 51.97%

Top 5 barrios originales que NO coincidieron:
barrio
SUBA          32
CASTILLA      27
EL PRADO      24
TINTAL SUR    22
TIBABUYES     21
Name: count, dtype: int64


In [9]:
# Asumiendo que df_merged es el DataFrame después del último merge
propiedades_sin_localidad = df_merged[df_merged['Localidad'].isna()]

# Cuenta los 10 nombres de barrio únicos más comunes que fallaron
barrios_unicos_faltantes = propiedades_sin_localidad['barrio_limpio'].value_counts().head(10)

print("Top 10 Nombres de Barrio Únicos que Causan la Mayoría de Faltantes:")
print(barrios_unicos_faltantes)

Top 10 Nombres de Barrio Únicos que Causan la Mayoría de Faltantes:
barrio_limpio
SUBA               32
CASTILLA           27
EL PRADO           24
TINTAL SUR         22
TIBABUYES          21
CIUDAD USME        21
LA ALHAMBRA        14
USAQUEN            14
ENGATIVA           13
BOSA OCCIDENTAL    12
Name: count, dtype: int64


In [10]:
# Ejemplo de correcciones que podrías necesitar
mapeo_correccion_manual = {
    'SANTAFE': 'SANTA FE',
    'CHICO NORTE': 'CHICO NORTE II', # O el nombre más cercano en tu ref.
    'LA CASTELLANA': 'CASTELLANA',
    'CASA BLANCA': 'CASA BLANCA BAJA' # Etc.
}

In [11]:
df_merged['barrio_limpio'] = df_merged['barrio_limpio'].replace(mapeo_correccion_manual)

In [12]:
# Imputar los valores nulos en 'Localidad' y 'UPZ' con 'DESCONOCIDO' o 'FUERA DE RANGO'
df_merged['Localidad'] = df_merged['Localidad'].fillna('DESCONOCIDO')
df_merged['UPZ'] = df_merged['UPZ'].fillna('DESCONOCIDO')

# Ahora, la columna 'Localidad' no tiene NaN y está lista para el One-Hot Encoding

In [13]:
propiedades_sin_localidad = df_merged['Localidad'].isna().sum()

print(f"\nNúmero de propiedades sin Localidad asignada (NaN): {propiedades_sin_localidad}")
print(f"El porcentaje de datos sin unión es: {(propiedades_sin_localidad / len(df_merged) * 100):.2f}%")

# Opcional: ver los nombres de los barrios originales que fallaron
barrios_faltantes_frecuentes = df_merged[df_merged['Localidad'].isna()]['barrio'].value_counts().head(5)
print("\nTop 5 barrios originales que NO coincidieron:")
print(barrios_faltantes_frecuentes)


Número de propiedades sin Localidad asignada (NaN): 0
El porcentaje de datos sin unión es: 0.00%

Top 5 barrios originales que NO coincidieron:
Series([], Name: count, dtype: int64)


In [15]:
df_merged.head()

,conjunto,administración,estrato,antiguedad,remodelado,área,habitaciones,baños,garajes,elevadores,...,gas,parqueadero,precio,direccion,nombre,descripcion,barrio,barrio_limpio,Localidad,UPZ
0,Santa Mónica,532000,4.0,37,Si,86.0,1,1,1,1,...,Si,No,313900000.0,Avenida Carrera 3 # 59-65,Santa Mónica,"Apartamento en venta de 86 m2, con vista exter...",PARDO RUBIO,PARDO RUBIO,Chapinero,Pardo Rubio
1,Chicó Milano 101,0,6.0,7,Si,77.0,1,2,2,1,...,Si,Si,440100000.0,Carrera 12 #101A-18,Chicó Milano 101,"Apartamento en venta de 77m2, con vista interi...",USAQUEN,USAQUEN,DESCONOCIDO,DESCONOCIDO
2,Portal del Belmira,811893,4.0,14,Si,109.0,3,4,2,1,...,Si,Si,495000000.0,Calle 146 #7F-54,Portal del Belmira,"Apartamento en venta de 109 m2, con vista Inte...",LOS CEDROS,LOS CEDROS,Usaquén,Los Cedros
3,Carmel Reservado,400200,4.0,11,Si,76.0,3,2,1,2,...,Si,Si,442300000.0,KR 54 152A 35,Carmel Reservado,"Apartamento en venta de 76m2, con vista exteri...",EL PRADO,EL PRADO,DESCONOCIDO,DESCONOCIDO
4,Toscana 1,270000,4.0,20,Si,105.0,4,2,1,0,...,Si,Si,387000000.0,CL 168 14 55,Toscana 1,"Casa en venta de 105m2, con vista exterior, ub...",TOBERIN,TOBERIN,DESCONOCIDO,DESCONOCIDO


In [19]:
import pandas as pd
import unicodedata

# --- 1. Definir Función de Limpieza ---
# Se necesita para crear la llave de unión 'barrio_limpio'
def limpiar_texto(s):
    if isinstance(s, str):
        s = s.strip().upper()
        # Normaliza para eliminar tildes/caracteres diacríticos
        s = ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')
        return s
    return s

# --- 2. Cargar los DataFrames ---
# Asegúrate de usar el delimitador correcto si aplica (properties.csv parece usar coma)
df_properties = pd.read_csv('properties.csv')
df_barrios_ref = pd.read_csv('barrios_bogota_localidades_limpio2.csv')

# --- 3. Preparación y Merge ---
# Crear la llave de unión limpia en ambas bases
df_properties['barrio_limpio'] = df_properties['barrio'].apply(limpiar_texto)
df_barrios_ref['barrio_limpio'] = df_barrios_ref['Barrio'].apply(limpiar_texto)

# Eliminar duplicados en la base de referencia (para evitar errores en el merge)
df_barrios_ref_unique = df_barrios_ref[['Localidad', 'UPZ', 'barrio_limpio']].drop_duplicates(subset=['barrio_limpio'])

# Realizar el Left Merge
df_merged = df_properties.merge(
    df_barrios_ref_unique,
    left_on='barrio_limpio',
    right_on='barrio_limpio',
    how='left'
)

# --- 4. Imprimir el Head del DataFrame final ---
print("Head del DataFrame 'df_merged' (primeras 5 filas):")
df_merged.head()

Head del DataFrame 'df_merged' (primeras 5 filas):


,conjunto,administración,estrato,antiguedad,remodelado,área,habitaciones,baños,garajes,elevadores,...,gas,parqueadero,precio,direccion,nombre,descripcion,barrio,barrio_limpio,Localidad,UPZ
0,Santa Mónica,532000,4.0,37,Si,86.0,1,1,1,1,...,Si,No,313900000.0,Avenida Carrera 3 # 59-65,Santa Mónica,"Apartamento en venta de 86 m2, con vista exter...",PARDO RUBIO,PARDO RUBIO,Chapinero,Pardo Rubio
1,Chicó Milano 101,0,6.0,7,Si,77.0,1,2,2,1,...,Si,Si,440100000.0,Carrera 12 #101A-18,Chicó Milano 101,"Apartamento en venta de 77m2, con vista interi...",USAQUEN,USAQUEN,NaN,NaN
2,Portal del Belmira,811893,4.0,14,Si,109.0,3,4,2,1,...,Si,Si,495000000.0,Calle 146 #7F-54,Portal del Belmira,"Apartamento en venta de 109 m2, con vista Inte...",LOS CEDROS,LOS CEDROS,Usaquén,Los Cedros
3,Carmel Reservado,400200,4.0,11,Si,76.0,3,2,1,2,...,Si,Si,442300000.0,KR 54 152A 35,Carmel Reservado,"Apartamento en venta de 76m2, con vista exteri...",EL PRADO,EL PRADO,NaN,NaN
4,Toscana 1,270000,4.0,20,Si,105.0,4,2,1,0,...,Si,Si,387000000.0,CL 168 14 55,Toscana 1,"Casa en venta de 105m2, con vista exterior, ub...",TOBERIN,TOBERIN,NaN,NaN


In [26]:
# Asumimos que df_merged ya existe y contiene las columnas 'barrio_limpio' y 'Localidad'.

# 1. Crear un DataFrame solo con las filas que tienen Localidad faltante (NaN)
df_faltantes = df_merged[df_merged['Localidad'].isna()]

# 2. Contar la frecuencia de los nombres de barrio limpios en ese subconjunto
# Esto nos dice qué barrios (ya limpios) causan la mayoría de los NaN.
barrios_unicos_faltantes = df_faltantes['barrio_limpio'].value_counts()

# 3. Mostrar el Top 10 o Top 20 para priorizar la corrección
print("Top 20 Nombres de Barrio Únicos sin Localidad Asignada:")
print(barrios_unicos_faltantes.head(20))

# Para referencia: número total de propiedades faltantes
propiedades_sin_localidad = len(df_faltantes)
print(f"\nTotal de propiedades sin Localidad: {propiedades_sin_localidad}")

Top 20 Nombres de Barrio Únicos sin Localidad Asignada:
Series([], Name: count, dtype: int64)

Total de propiedades sin Localidad: 0


In [ ]:
import pandas as pd
import unicodedata

# 1. Definición del Mapeo Manual (Usando tus correcciones)
# Se usa la versión limpia del barrio (MAYÚSCULAS, sin tildes)
mapeo_localidad_manual = {
    'SUBA': 'SUBA',
    'CASTILLA': 'KENNEDY',
    'EL PRADO': 'SUBA',
    'EL TINTAL': 'KENNEDY',
    'TIBABUYES': 'SUBA',
    'CIUDAD USME': 'USME', # Corregido: 'ciuidad usme' -> 'CIUDAD USME'
    'LA ALHAMBRA': 'SUBA',
    'USAQUEN': 'USAQUEN',
    'ENGATIVA': 'ENGATIVA',
    'BOSA OCCIDENTAL': 'BOSA',
    'BOSA CENTRAL': 'BOSA',
    'EL RINCON': 'SUBA',
    'VERBENAL': 'USAQUEN',
    'FONTIBON SAN PABLO': 'FONTIBON', # Corregido: espacios y tildes
    'TOBERIN': 'USAQUEN',
    'AMERICAS': 'KENNEDY',
    'CHICO LAGO': 'CHAPINERO', # Corregido: 'chapinero' -> 'CHAPINERO'
    'FONTIBON': 'FONTIBON',
    'SOSIEGO': 'SAN CRISTOBAL', # Corregido: 'san cristobal' -> 'SAN CRISTOBAL'
    'TINTAL NORTE': 'KENNEDY' # Corregido: 'Tintal norte' -> 'TINTAL NORTE'
}

# 2. Aplicar el mapeo a las filas faltantes

# A. Crear la columna temporal con las correcciones
# El método .map() usa la columna 'barrio_limpio' para buscar la Localidad en el diccionario.
df_merged['Localidad_corregida'] = df_merged['barrio_limpio'].map(mapeo_localidad_manual)

# B. Consolidar la columna 'Localidad' (el paso crucial)
# Si 'Localidad' (original) es NaN, toma el valor de 'Localidad_corregida'.
# Si 'Localidad' (original) tiene un valor, lo mantiene.
df_merged['Localidad'] = df_merged['Localidad'].fillna(df_merged['Localidad_corregida'])

# C. Eliminar la columna temporal
df_merged = df_merged.drop(columns=['Localidad_corregida'])

print("Corrección manual de Localidades completada.")

Corrección manual de Localidades completada.


In [25]:
# Contar el número de propiedades sin localidad asignada
propiedades_final_faltantes = df_merged['Localidad'].isna().sum()

print("\n--- RESULTADOS DE LA CORRECCIÓN ---")
print(f"Total de propiedades con Localidad Faltante (NaN) después de la corrección: {propiedades_final_faltantes}")

# Si aún quedan faltantes, revisa los nuevos top 5 que causan el error:
if propiedades_final_faltantes > 0:
    df_faltantes_final = df_merged[df_merged['Localidad'].isna()]
    barrios_unicos_faltantes_final = df_faltantes_final['barrio_limpio'].value_counts().head(5)
    print("\nTop 5 Barrios que AÚN FALTAN (si los hay):")
    print(barrios_unicos_faltantes_final)
else:
    print("\n¡Todas las propiedades del Top 20 han sido asignadas con éxito!")

# Imputación final de los restantes
# Reemplazar los NaN restantes con 'OTROS' o 'DESCONOCIDO' para que no afecten el modelo.
df_merged['Localidad'] = df_merged['Localidad'].fillna('DESCONOCIDO')
print("\nLos NaN restantes fueron imputados con 'DESCONOCIDO'. La columna 'Localidad' está lista.")


--- RESULTADOS DE LA CORRECCIÓN ---
Total de propiedades con Localidad Faltante (NaN) después de la corrección: 0

¡Todas las propiedades del Top 20 han sido asignadas con éxito!

Los NaN restantes fueron imputados con 'DESCONOCIDO'. La columna 'Localidad' está lista.


In [27]:
df_merged.head(50)

,conjunto,administración,estrato,antiguedad,remodelado,área,habitaciones,baños,garajes,elevadores,...,gas,parqueadero,precio,direccion,nombre,descripcion,barrio,barrio_limpio,Localidad,UPZ
0,Santa Mónica,532000,4.0,37,Si,86.0,1,1,1,1,...,Si,No,313900000.0,Avenida Carrera 3 # 59-65,Santa Mónica,"Apartamento en venta de 86 m2, con vista exter...",PARDO RUBIO,PARDO RUBIO,Chapinero,Pardo Rubio
1,Chicó Milano 101,0,6.0,7,Si,77.0,1,2,2,1,...,Si,Si,440100000.0,Carrera 12 #101A-18,Chicó Milano 101,"Apartamento en venta de 77m2, con vista interi...",USAQUEN,USAQUEN,USAQUEN,NaN
2,Portal del Belmira,811893,4.0,14,Si,109.0,3,4,2,1,...,Si,Si,495000000.0,Calle 146 #7F-54,Portal del Belmira,"Apartamento en venta de 109 m2, con vista Inte...",LOS CEDROS,LOS CEDROS,Usaquén,Los Cedros
3,Carmel Reservado,400200,4.0,11,Si,76.0,3,2,1,2,...,Si,Si,442300000.0,KR 54 152A 35,Carmel Reservado,"Apartamento en venta de 76m2, con vista exteri...",EL PRADO,EL PRADO,SUBA,NaN
4,Toscana 1,270000,4.0,20,Si,105.0,4,2,1,0,...,Si,Si,387000000.0,CL 168 14 55,Toscana 1,"Casa en venta de 105m2, con vista exterior, ub...",TOBERIN,TOBERIN,USAQUEN,NaN
5,Rondas de San Patricio 2,211000,3.0,19,Si,67.0,3,2,1,0,...,Si,Si,271500000.0,Calle 80A #118-30,Rondas de San Patricio 2,"Apartamento en venta de 67 m2, con vista Inter...",BOLIVIA,BOLIVIA,Engativá,Bo
6,Prados de Capellanía 3,226000,4.0,24,Si,72.0,3,2,1,2,...,Si,Si,334000000.0,KR 89 # 19 A - 50,Prados de Capellanía 3,"Apartamento en venta de 72m2, con vista interi...",GRANJAS DE TECHO,GRANJAS DE TECHO,Fontibón,Granjas de
7,Edificio El Campin,529000,4.0,30,Si,77.0,2,2,1,1,...,Si,No,444200000.0,Calle 55 # 30 - 27,Edificio El Campin,"Apartamento en venta de 77m2, con vista exteri...",LA ESMERALDA,LA ESMERALDA,Usme,Alfonso López
8,Tierra Del Sol,130000,3.0,9,Si,59.0,3,2,1,1,...,Si,Si,256800000.0,Calle 6b # 81b-51,Tierra Del Sol,"Apartamento en venta de 59 m2, con vista exter...",CASTILLA,CASTILLA,DESCONOCIDO,NaN
9,Edificio Omega 134,453000,5.0,8,Si,54.0,1,2,1,1,...,Si,Si,384200000.0,Carrera 23#134A-09,Edificio Omega 134,"Apartamento en venta de 60m2, con vista exteri...",LOS CEDROS,LOS CEDROS,Usaquén,Los Cedros


In [28]:
import pandas as pd
import unicodedata

# 1. Función de limpieza (la misma que hemos usado)
def limpiar_texto(s):
    """Estandariza el texto: mayúsculas, sin tildes, sin espacios iniciales/finales."""
    if isinstance(s, str):
        s = s.strip().upper()
        # Normaliza para eliminar tildes/caracteres diacríticos
        s = ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')
        return s
    return s

# 2. Asumiendo que tu DataFrame final se llama df_merged

# Aplica la limpieza a la columna 'Localidad'. 
# Esto convertirá 'Usaquén' en 'USAQUEN' y 'Engativá' en 'ENGATIVA', etc.
df_merged['Localidad'] = df_merged['Localidad'].apply(limpiar_texto)

print("La columna 'Localidad' ha sido limpiada (MAYÚSCULAS y SIN TILDES).")

La columna 'Localidad' ha sido limpiada (MAYÚSCULAS y SIN TILDES).


In [ ]:
print("\nValores únicos de la columna 'Localidad' después de la limpieza:")
# Utiliza .value_counts() para ver la lista de localidades y el número de propiedades en cada una.
print(df_merged['Localidad'].value_counts())


Valores únicos de la columna 'Localidad' después de la limpieza:
Localidad
SUBA                  143
USAQUEN               108
DESCONOCIDO            79
KENNEDY                67
USME                   46
ENGATIVA               40
FONTIBON               23
BOSA                   23
CIUDAD BOLIVAR         18
CHAPINERO              15
SAN CRISTOBAL           5
PUENTE ARANDA           5
TEUSAQUILLO             4
RAFAEL URIBE URIBE      4
LOS MARTIRES            2
BARRIOS UNIDOS          2
ANTONIO NARINO          1
Name: count, dtype: int64


In [31]:
# Asumiendo que tu DataFrame se llama df_merged y que ya aplicaste la limpieza de tildes
# (Esto incluye los NaN que imputaste con 'DESCONOCIDO' en el paso anterior).

# Contar cuántas propiedades tienen la etiqueta 'DESCONOCIDO'
conteo_desconocidos = df_merged[df_merged['Localidad'] == 'DESCONOCIDO'].shape[0]

# Contar cuántas propiedades aún tienen NaN (si no se aplicó el fillna)
conteo_nan = df_merged['Localidad'].isna().sum()

print(f"Propiedades con Localidad = 'DESCONOCIDO': {conteo_desconocidos}")
print(f"Propiedades con Localidad = NaN: {conteo_nan}")

# Si tienes un conteo alto en 'DESCONOCIDO' (el valor que usaste para imputar), todo está correcto.

Propiedades con Localidad = 'DESCONOCIDO': 79
Propiedades con Localidad = NaN: 0


In [34]:
# Asumimos que df_merged es tu DataFrame actual y que ya se aplicó:
# 1. El merge inicial.
# 2. La corrección manual de los barrios más comunes (SUBA, CASTILLAS, etc.).
# 3. La limpieza de tildes en la columna 'Localidad'.
# 4. La imputación de los NaN restantes con 'DESCONOCIDO'.

# 1. Crear un DataFrame solo con las filas que tienen Localidad como 'DESCONOCIDO'
df_desconocidos = df_merged[df_merged['Localidad'] == 'DESCONOCIDO']

# 2. Contar la frecuencia de los nombres de barrio limpios en ese subconjunto
# Esto nos dice qué barrios (ya limpios) son los que más fallaron al mapear.
barrios_unicos_desconocidos = df_desconocidos['barrio_limpio'].value_counts()

# 3. Mostrar el Top 20 de los barrios que necesitan corrección
print("Top 20 Nombres de Barrio Únicos con Localidad 'DESCONOCIDO':")
print(barrios_unicos_desconocidos.head(50))

# Para referencia: número total de propiedades DESCONOCIDAS
propiedades_desconocidas = len(df_desconocidos)
print(f"\nTotal de propiedades con Localidad 'DESCONOCIDO': {propiedades_desconocidas}")

Top 20 Nombres de Barrio Únicos con Localidad 'DESCONOCIDO':
barrio_limpio
CASTILLA                   27
TINTAL SUR                 22
APOGEO                      4
ARBORIZADORA                4
BOYACA REAL                 4
MODELIA                     4
LA FLORESTA                 4
CHAPINERO                   3
CAPELLANIA                  2
BOSA 61                     1
COMUNEROS                   1
CIUDAD SALITRE ORIENTAL     1
PATIO BONITO                1
LOS ALCAZARES               1
Name: count, dtype: int64

Total de propiedades con Localidad 'DESCONOCIDO': 79


In [35]:
import pandas as pd

# 1. Definición del Mapeo Manual Adicional (Usando tus nuevas correcciones)
# Se usa la versión limpia del barrio (MAYÚSCULAS, sin tildes) y las localidades en MAYÚSCULAS.
mapeo_adicional = {
    'CASTILLA': 'KENNEDY',
    'TINTAL SUR': 'KENNEDY',
    'APOGEO': 'BOSA',
    'ARBORIZADORA': 'CIUDAD BOLIVAR',
    'BOYACA REAL': 'ENGATIVA',
    'MODELIA': 'FONTIBON',
    'LA FLORESTA': 'SUBA',
    'CHAPINERO': 'CHAPINERO',
    'CAPELLANIA': 'FONTIBON',
    'BOSA 61': 'BOSA',
    'COMUNEROS': 'SUBA',
    'CIUDAD SALITRE ORIENTAL': 'TEUSAQUILLO',
    'PATIO BONITO': 'KENNEDY',
    'LOS ALCAZARES': 'BARRIOS UNIDOS'
}

# 2. Aplicar el mapeo a las filas faltantes

# A. Crear la columna temporal con las nuevas correcciones
# El método .map() utiliza 'barrio_limpio' para buscar la Localidad en el diccionario.
df_merged['Localidad_corregida_2'] = df_merged['barrio_limpio'].map(mapeo_adicional)

# B. Consolidar la columna 'Localidad' (paso crucial)
# Los valores en 'Localidad' que son 'DESCONOCIDO' (o NaN) serán reemplazados por el valor de 'Localidad_corregida_2'
# Usamos .replace('DESCONOCIDO', pd.NA) para poder usar fillna, que es más seguro.
df_merged['Localidad'] = df_merged['Localidad'].replace('DESCONOCIDO', pd.NA).fillna(df_merged['Localidad_corregida_2'])

# C. Volver a imputar los NaN restantes con 'DESCONOCIDO'
df_merged['Localidad'] = df_merged['Localidad'].fillna('DESCONOCIDO')


# D. Eliminar la columna temporal
df_merged = df_merged.drop(columns=['Localidad_corregida_2'])

print("Corrección manual de Localidades (fase 2) completada.")

Corrección manual de Localidades (fase 2) completada.


In [36]:
# Contar el número final de propiedades sin localidad asignada
propiedades_final_desconocidas = df_merged[df_merged['Localidad'] == 'DESCONOCIDO'].shape[0]

print("\n--- RESULTADOS DE LA CORRECCIÓN FINAL ---")
print(f"Propiedades restantes con Localidad 'DESCONOCIDO': {propiedades_final_desconocidas}")

# Si quedan pendientes (generalmente menos de 100), revisa los nuevos top 5:
if propiedades_final_desconocidas > 0:
    df_faltantes_final = df_merged[df_merged['Localidad'] == 'DESCONOCIDO']
    barrios_unicos_faltantes_final = df_faltantes_final['barrio_limpio'].value_counts().head(5)
    print("\nTop 5 Barrios que AÚN FALTAN (si los hay):")
    print(barrios_unicos_faltantes_final)
else:
    print("\n¡Ya no quedan barrios pendientes del Top 20! La limpieza de localidad está casi completa.")


--- RESULTADOS DE LA CORRECCIÓN FINAL ---
Propiedades restantes con Localidad 'DESCONOCIDO': 0

¡Ya no quedan barrios pendientes del Top 20! La limpieza de localidad está casi completa.


In [37]:
df_merged.head(50
               )

,conjunto,administración,estrato,antiguedad,remodelado,área,habitaciones,baños,garajes,elevadores,...,gas,parqueadero,precio,direccion,nombre,descripcion,barrio,barrio_limpio,Localidad,UPZ
0,Santa Mónica,532000,4.0,37,Si,86.0,1,1,1,1,...,Si,No,313900000.0,Avenida Carrera 3 # 59-65,Santa Mónica,"Apartamento en venta de 86 m2, con vista exter...",PARDO RUBIO,PARDO RUBIO,CHAPINERO,Pardo Rubio
1,Chicó Milano 101,0,6.0,7,Si,77.0,1,2,2,1,...,Si,Si,440100000.0,Carrera 12 #101A-18,Chicó Milano 101,"Apartamento en venta de 77m2, con vista interi...",USAQUEN,USAQUEN,USAQUEN,NaN
2,Portal del Belmira,811893,4.0,14,Si,109.0,3,4,2,1,...,Si,Si,495000000.0,Calle 146 #7F-54,Portal del Belmira,"Apartamento en venta de 109 m2, con vista Inte...",LOS CEDROS,LOS CEDROS,USAQUEN,Los Cedros
3,Carmel Reservado,400200,4.0,11,Si,76.0,3,2,1,2,...,Si,Si,442300000.0,KR 54 152A 35,Carmel Reservado,"Apartamento en venta de 76m2, con vista exteri...",EL PRADO,EL PRADO,SUBA,NaN
4,Toscana 1,270000,4.0,20,Si,105.0,4,2,1,0,...,Si,Si,387000000.0,CL 168 14 55,Toscana 1,"Casa en venta de 105m2, con vista exterior, ub...",TOBERIN,TOBERIN,USAQUEN,NaN
5,Rondas de San Patricio 2,211000,3.0,19,Si,67.0,3,2,1,0,...,Si,Si,271500000.0,Calle 80A #118-30,Rondas de San Patricio 2,"Apartamento en venta de 67 m2, con vista Inter...",BOLIVIA,BOLIVIA,ENGATIVA,Bo
6,Prados de Capellanía 3,226000,4.0,24,Si,72.0,3,2,1,2,...,Si,Si,334000000.0,KR 89 # 19 A - 50,Prados de Capellanía 3,"Apartamento en venta de 72m2, con vista interi...",GRANJAS DE TECHO,GRANJAS DE TECHO,FONTIBON,Granjas de
7,Edificio El Campin,529000,4.0,30,Si,77.0,2,2,1,1,...,Si,No,444200000.0,Calle 55 # 30 - 27,Edificio El Campin,"Apartamento en venta de 77m2, con vista exteri...",LA ESMERALDA,LA ESMERALDA,USME,Alfonso López
8,Tierra Del Sol,130000,3.0,9,Si,59.0,3,2,1,1,...,Si,Si,256800000.0,Calle 6b # 81b-51,Tierra Del Sol,"Apartamento en venta de 59 m2, con vista exter...",CASTILLA,CASTILLA,KENNEDY,NaN
9,Edificio Omega 134,453000,5.0,8,Si,54.0,1,2,1,1,...,Si,Si,384200000.0,Carrera 23#134A-09,Edificio Omega 134,"Apartamento en venta de 60m2, con vista exteri...",LOS CEDROS,LOS CEDROS,USAQUEN,Los Cedros


In [38]:
print("\nValores únicos de la columna 'Localidad' después de la limpieza:")
# Utiliza .value_counts() para ver la lista de localidades y el número de propiedades en cada una.
print(df_merged['Localidad'].value_counts())


Valores únicos de la columna 'Localidad' después de la limpieza:
Localidad
SUBA                  148
KENNEDY               117
USAQUEN               108
USME                   46
ENGATIVA               44
FONTIBON               29
BOSA                   28
CIUDAD BOLIVAR         22
CHAPINERO              18
TEUSAQUILLO             5
PUENTE ARANDA           5
SAN CRISTOBAL           5
RAFAEL URIBE URIBE      4
BARRIOS UNIDOS          3
LOS MARTIRES            2
ANTONIO NARINO          1
Name: count, dtype: int64


In [39]:
import pandas as pd

# Asumiendo que df_merged es el DataFrame final después de todas las correcciones.

# 1. Contar el número de propiedades restantes con Localidad 'DESCONOCIDO'
conteo_desconocidos = df_merged[df_merged['Localidad'] == 'DESCONOCIDO'].shape[0]

# 2. Contar si, por alguna razón, quedaron valores NaN
conteo_nan = df_merged['Localidad'].isna().sum()

# 3. Contar el número total de propiedades
total_propiedades = len(df_merged)

print("--- Resumen de la Limpieza de Localidad ---")
print(f"Total de Propiedades: {total_propiedades}")
print("-" * 35)
print(f"Propiedades restantes con Localidad 'DESCONOCIDO': {conteo_desconocidos}")
print(f"Propiedades restantes con Localidad NaN: {conteo_nan}")
print("-" * 35)

# 4. Imprimir la distribución de todas las localidades asignadas
print("\nDistribución de Propiedades por Localidad (Top 10):")
print(df_merged['Localidad'].value_counts().head(10))

--- Resumen de la Limpieza de Localidad ---
Total de Propiedades: 585
-----------------------------------
Propiedades restantes con Localidad 'DESCONOCIDO': 0
Propiedades restantes con Localidad NaN: 0
-----------------------------------

Distribución de Propiedades por Localidad (Top 10):
Localidad
SUBA              148
KENNEDY           117
USAQUEN           108
USME               46
ENGATIVA           44
FONTIBON           29
BOSA               28
CIUDAD BOLIVAR     22
CHAPINERO          18
TEUSAQUILLO         5
Name: count, dtype: int64
